In [20]:
import sys
sys.path.append('../')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import os
import scipy.sparse as sp
from time import time
import math
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score,accuracy_score,precision_score,recall_score,multilabel_confusion_matrix,roc_curve,roc_auc_score
from pytorchtools import EarlyStopping  #location: /utils/
from utils import sparse_mx_to_torch_sparse_tensor,accuracy
from layers import GraphConvolution, GraphAttention
from tqdm import tqdm

In [21]:
class Graph():
    def __init__(self,num_nodes,edge_pairs,features,label,node_degs):
        self.num_nodes = num_nodes
        self.edge_pairs = edge_pairs
        self.features = features
        self.label = label
        self.node_degs = node_degs  #邻接点数量
        
    def __str__(self):
        return ('nodes: %d  edge_pairs: %d features: %d' % (self.num_nodes,len(self.edge_pairs),len(self.features)))

In [22]:
def getGraph(filename):
    f = open(filename)
    row = f.readline().strip().split()
    nodes,label = [int(w) for w in row]
    node_features = []
    edge_pairs = []
    node_degs = []
    for j in range(nodes):
        row = f.readline().strip().split()
        node_deg = int(row[0])+1
        row ,attr = [int(w) for w in row[1:int(row[0])+1]],np.array([float(w) for w in row[int(row[0])+1:]])
        if attr is not None:
            node_features.append(attr)
        if node_deg is not None:
            node_degs.append(node_deg)
        if row is not None:
            for k in row:
                edge = [j,k]
                edge_pairs.append(edge)
    g = Graph(nodes,edge_pairs,node_features,label,node_degs)
    return g

In [23]:
def getGraphList(files):
    glist = []
    for file in files:
        graph = getGraph(file)
        glist.append(graph)
    return glist

In [24]:
def getEdgeList(files,k):
    EdgeList = []
    for file in files:
        li = np.loadtxt(file)
        EdgeList.append(li)
    EdgeList = processEdgeList(EdgeList,k)
    return EdgeList

def processEdgeList(edges,k):
    newList = []
    for item in edges:
        if item.shape[0] >= k:
            newList.append(item[:k,:].tolist())
        else:
            t = np.zeros((k-item.shape[0],100))
            newList.append(np.concatenate((item,t)).tolist())
    return torch.FloatTensor(np.array(newList))


def getEdgeNum(files):
    EdgeNum = []
    for file in files:
        li = np.loadtxt(file)
        EdgeNum.append(li.shape[0])
    return EdgeNum

In [25]:
# te = getEdgeList(train_edge_data[:5],99)
# te = torch.FloatTensor(te)
# mo = Edge_Attention(2,100)
# a = mo(te)
# encoder_layer = nn.TransformerEncoderLayer(d_model=100, nhead=2)
# src = torch.rand(10, 32, 512)
# out = encoder_layer(te)

In [26]:
# a = [1,2,3,4]
# b = [1,2,6,4]
# (np.array(a)+np.array(b))/2

In [27]:
def merge_Graph(graph_list):
    prefix_sum = []
    node_features = []
    node_degs = []
    node_labels = []
    total_num_edges = 0
    total_num_nodes = 0
    edge_pairs = []
    graph_sizes = []
    for i in range(len(graph_list)):
        prefix_sum.append(graph_list[i].num_nodes)
        if i != 0:
            prefix_sum[i] += prefix_sum[i-1]
        node_features.extend(graph_list[i].features)
        node_degs.extend(graph_list[i].node_degs)
        node_labels.append(graph_list[i].label)
        total_num_edges += len(graph_list[i].edge_pairs)
        total_num_nodes += graph_list[i].num_nodes
        graph_sizes.append(graph_list[i].num_nodes)
        edge_pairs.append(graph_list[i].edge_pairs)
    # create batch_graph
    n2n_idxes = torch.LongTensor(2, total_num_edges)
    n2n_vals = torch.FloatTensor(total_num_edges)
    
    for i in range(len(graph_list)):
        prefix_sum[len(graph_list)-i-1] = prefix_sum[len(graph_list)-i-2]
    prefix_sum[0] = 0
    
    for i in range(total_num_edges):
        n2n_vals[i] = 1
    
    j = 0
    for i in range(len(graph_list)):
        for item in edge_pairs[i]:
            n2n_idxes[0][j] = item[0]+prefix_sum[i]
            n2n_idxes[1][j] = item[1]+prefix_sum[i]
            
#             if item[0]+prefix_sum[i] > total_num_nodes:
#                 print('item0',item[0],prefix_sum[i],total_num_nodes)
#             if item[1]+prefix_sum[i] > total_num_nodes:
#                 print('item1',item[1],prefix_sum[i],total_num_nodes)
                
#             if item[0]+prefix_sum[i] < 0:
#                 print('item0',item[0],prefix_sum[i],'position: ',i)
#             if item[1]+prefix_sum[i] < 0 :
#                 print('item1',item[1],prefix_sum[i],'position: ',i)
                
            j += 1
#     print(j,total_num_edges)    
#     print(n2n_idxes[:,3000:])
#     print(node_features)
    n2n = torch.sparse.FloatTensor(n2n_idxes, n2n_vals, torch.Size([total_num_nodes, total_num_nodes]))
    node_features = torch.FloatTensor(node_features)
    node_degs = 1/torch.LongTensor(node_degs)    
    degs_index = torch.LongTensor(2, total_num_nodes)
    
    for i in range(total_num_nodes):
        degs_index[0,i] = i
        degs_index[1,i] = i
    node_degs = torch.sparse.FloatTensor(degs_index, node_degs, torch.Size([total_num_nodes, total_num_nodes]))
        
    return n2n,node_features,node_degs,graph_sizes
    

In [28]:
def getInverse(adjs):
    Dres = []
    for a in adjs:
        length = a.size()[0]
        D = np.zeros((length,length))
        for item,i in zip(a,range(length)):
            if item.sum() != 0:
                D[i,i] = 1/item.sum()
        Dres.append(D)
    return torch.tensor(np.array(Dres,dtype = np.float32)).cuda()

In [29]:
class GraphConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, activation = None, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.activation = activation
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, features, adj, degs):
        support = torch.mm(features, self.weight)
        output = torch.mm(adj, support)
        output = torch.mm(degs,output)
        if self.activation != None:
            output = self.activation(output)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [30]:
class Edge_Attention(nn.Module):
    def __init__(self, head, feature_length):
        super(Edge_Attention,self).__init__()
        self.head = head
        self.feature_length = feature_length
        self.Attention = nn.TransformerEncoderLayer(d_model=feature_length, nhead=head, batch_first=True)
        
    def forward(self,edges):
        return self.Attention(edges)
    

In [31]:
# te = getGraphList(train_graph_data[:5])
# n2n,node_features,node_degs,graph_sizes = merge_Graph(te)
# mo = res_GCN(100,32,1,5,99)
# temp = mo(node_features,n2n,node_degs,graph_sizes)
# temp.shape

In [32]:
class res_GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, n_layers, k):
        super(res_GCN, self).__init__()

        self.k = k
        self.n_layers = n_layers
        self.nhid = nhid
        self.nclass = nclass
        self.layers = nn.ModuleList()
        self.total_latent_dim = nhid * n_layers + nclass
        # 输入层
        self.layers.append( GraphConvolution( nfeat, nhid, activation = torch.tanh ))
        # 隐层
        for i in range(n_layers - 1):
            self.layers.append(GraphConvolution(nhid, nhid, activation = torch.tanh ))
        # 输出层
        self.layers.append( GraphConvolution(nhid, nclass))
        
        
    def forward(self, features, graphs,degs,graph_sizes):
        
        # def a res GCNN here
        feature_list = []
        for i, layer in enumerate( self.layers):
            if i!= 0 and i != len(self.layers)-1:
                features = layer( features, graphs, degs) + features
                feature_list.append(features)
            else:
                features = layer( features, graphs, degs)
                feature_list.append(features)
#         sort pooling with k row remains

        res = ''
        for i,item in enumerate(feature_list):
            if i == 0:
                res = item
            else:
                res = torch.cat((res,item),1)
                
#         res = features

#         print(len(feature_list),res.shape)
        sort_channel = res[:, -1]
        batch_sortpooling_graphs = torch.zeros(len(graph_sizes), self.k, self.total_latent_dim)
        if torch.cuda.is_available() and isinstance(features.data, torch.cuda.FloatTensor):
            batch_sortpooling_graphs = batch_sortpooling_graphs.cuda()

        batch_sortpooling_graphs = Variable(batch_sortpooling_graphs)
        accum_count = 0
        for i in range(len(graph_sizes)):
            to_sort = sort_channel[accum_count: accum_count + graph_sizes[i]]
            k = self.k if self.k <= graph_sizes[i] else graph_sizes[i]
            _, topk_indices = to_sort.topk(k)
            topk_indices += accum_count
            sortpooling_graph = res.index_select(0, topk_indices)
            if k < self.k:
                to_pad = torch.zeros(self.k-k, self.total_latent_dim)
                if torch.cuda.is_available() and isinstance(features.data, torch.cuda.FloatTensor):
                    to_pad = to_pad.cuda()

                to_pad = Variable(to_pad)
                sortpooling_graph = torch.cat((sortpooling_graph, to_pad), 0)
            batch_sortpooling_graphs[i] = sortpooling_graph
            accum_count += graph_sizes[i]

        return batch_sortpooling_graphs

In [33]:
12*32

384

In [34]:
class Classifier(nn.Module):
    def __init__(self,classNum,dropout_rate,nfeat,nhid,nclass,n_layers,k,head,features_length):
        super(Classifier,self).__init__()
        self.classNum = classNum
        self.dropout_rate = dropout_rate
        self.resGCNN = res_GCN(nfeat,nhid,nclass,n_layers,k)
        self.edgeAttention = Edge_Attention(head,features_length)
        
#         self.layer_norm1 = nn.LayerNorm()
        self.layer_norm2 = nn.LayerNorm(features_length)
        self.cov1 = nn.Conv1d(in_channels=2, out_channels=16,kernel_size = nhid*n_layers+nclass,stride = nhid*n_layers+nclass)
        self.maxpool = nn.MaxPool1d(kernel_size = 2, stride= 2)
        self.dropout1 = nn.Dropout(p = self.dropout_rate)
        self.cov2 = nn.Conv1d(in_channels=16, out_channels=32,kernel_size = 4,stride = 4)
        self.dropout2 = nn.Dropout(p = self.dropout_rate)
       
        dense = int(k/2/4)
        
        self.denseLayer1 = nn.Linear(dense*32,512)
        self.dropout3 = nn.Dropout(p = self.dropout_rate)
        self.denseLayer2 = nn.Linear(512,128)
        self.dropout4 = nn.Dropout(p = self.dropout_rate)
        self.outputLayer = nn.Linear(128,classNum)
        
    def forward(self,features, graphs,degs,graph_sizes,edges):
        gcn = self.resGCNN(features,graphs,degs,graph_sizes)
#         gcn = self.layer_norm1(gcn)
        e_attention = self.edgeAttention(edges)
        e_attention = self.layer_norm2(e_attention)
#         print(gcn.shape,e_attention.shape)
        
        gcn = gcn.view(len(graph_sizes),1,-1)
        e_attention = e_attention.view(len(graph_sizes),1,-1)
        
        
#         print(,gcn.shape,e_attention.shape)
        res = torch.cat((gcn,e_attention),1)
        
#         res = res.view(len(graph_sizes),1,-1)
#        1d convolution layer
        res = F.relu(self.cov1(res))
        res = self.dropout1(res)
        res = self.maxpool(res)
        res = F.relu(self.cov2(res))
        res = F.relu(res)
        res = self.dropout2(res)
#         Dense Layer 
        res = res.view(len(graph_sizes),-1)
        res = F.relu(self.denseLayer1(res))
        res = self.dropout3(res)
        res = F.relu(self.denseLayer2(res))
        res = self.dropout4(res)
        output = self.outputLayer(res)
        return output.flatten()
     

In [35]:
# a = torch.tensor([[0,1],[1,0]])
# a = a.unsqueeze(0)
# a.repeat((3,1,1))

In [36]:
# res = torch.randn(801,100)
# a = torch.tensor([[-0.9950, -0.6175, -0.1253,  1.3536],
#         [ 0.1208, -0.4237, -1.1313,  0.9022],
#         [-1.1995, -0.0699, -0.4396,  1.999]])
# # a,_  = torch.sort(a,dim=0,descending=True)
# key = [row[-1].item() for row in res]
# key = np.array(key)
# key = np.argsort(key,axis=-1,kind = 'quicksort')
# key = np.flipud(key)
# res = [list(res[i]) for i in key[:5]]
# res = torch.tensor(res)
# print(res.size())
# res = res.view(-1,1,5*100)
# print(res.size())

In [37]:
# this model process one graph per time
# have a try 
# 170000 files  shuffle and split into train test set
path = '../data/SemanticGraph/Graph_normal/'        # 无漏洞
pathA = '../data/SemanticGraph/Graph_vulnerable/'    # 有漏洞
files = os.listdir(path)
filesA = os.listdir(pathA)

e_path = '../data/SemanticGraph/BFS_Edges/Edge_no_v/'
e_pathA = '../data/SemanticGraph/BFS_Edges/Edge_v/'

edges = os.listdir(e_path)
edgesA = os.listdir(e_pathA)

graph_filename = [path+file for file in files]
graph_filename.extend([pathA+file for file in filesA])

edge_filename = [e_path+file for file in edges]
edge_filename.extend([e_pathA+file for file in edgesA])

all_label = [0.0 for i in range(len(files))]
all_label.extend([1.0 for i in range(len(filesA))]) 
length = len(files) + len(filesA)

# 打乱数据
from sklearn.utils import shuffle  
graph_filename,edge_filename,all_label = shuffle(graph_filename,edge_filename,all_label)

#划分数据集
k = 0.8
k1 = 0.9
train_graph_data = graph_filename[0:int(k*length)]
train_label = all_label[0:int(k*length)]
train_edge_data = edge_filename[0:int(k*length)]

valid_graph_data = graph_filename[int(k*length):int(k1*length)]
valid_label = all_label[int(k*length):int(k1*length)]
valid_edge_data = edge_filename[int(k*length):int(k1*length)]

test_graph_data = graph_filename[int(k1*length):]
test_label = all_label[int(k1*length):]
test_edge_data = edge_filename[int(k1*length):]

len(train_graph_data),len(valid_graph_data),len(test_graph_data)

(3200, 400, 400)

In [38]:
# def getGraphList(files):
#     glist = []
# #     Myfiles.extend(files)
#     for file in files:
#         graph = getGraph(file)
# #         print(graph)
# #         print(file)
#         glist.append(graph.num_nodes)
#     return glist

# a = getGraphList(graph_filename)
# b = np.sort(a)
# key = int(len(a)*0.9)
# print('90%: ',b[key])   # 116
# key = int(len(a)*0.8)
# print('80%: ',b[key])   # 116
# key = int(len(a)*0.6)
# print('60%: ',b[key])   # 78

# a = getEdgeNum(edge_filename)
# b = np.sort(a)
# k0 = int(len(a)*0.9) 
# k1 = int(len(a)*0.8)   
# k2 = int(len(a)*0.6)
# print('90%: ',b[k0],'80%: ',b[k1],"60%: ",b[k2])   # 80%:  115 60%:  69

In [39]:
# a = torch.FloatTensor([[0.5,0.3,0.2],[0.2,0.3,0.8]])
# b = torch.FloatTensor([[1,0,0],[0,1,1]])
# a = (a >= 0.5).float()
# # multilabel_confusion_matrix(b,a)
# c = (((a == b).float().sum(dim=1) / 3) == 1).sum()/a.shape[0]
# d = (a == b)[:,0].float().sum()
# d

In [40]:
# a = torch.tensor([[0.3711, 0.4049, 0.3462],
#         [0.3669, 0.4021, 0.3401],
#         [0.3930, 0.4236, 0.3792],
#         [0.3655, 0.4006, 0.3378],
#         [0.3577, 0.3937, 0.3271],
#         [0.3572, 0.3936, 0.3267],
#         [0.3711, 0.4049, 0.3462],
#         [0.3830, 0.4154, 0.3638],
#         [0.3887, 0.4202, 0.3726],
#         [0.3669, 0.4021, 0.3401],
#         [0.3705, 0.4047, 0.3452],
#         [0.3617, 0.3973, 0.3337]])
# a = a.cuda

In [41]:
def evaluate(predict,t_label,thresh):
    pre = (predict >= thresh).float()
#     print(predict,t_label)
    accuracy = accuracy_score(t_label.data.cpu(),pre.data.cpu())
    precision = precision_score(t_label.data.cpu(),pre.data.cpu(), zero_division = 0)
    recall = recall_score(t_label.data.cpu(),pre.data.cpu(), zero_division = 0)
    f1 = f1_score(t_label.data.cpu(),pre.data.cpu(), zero_division = 0)
    return precision, recall, f1, accuracy #without auc

def test_evaluate(predict,t_label,thresh):  
    pre = (predict >= thresh).float()
#     print(predict,t_label)
    accuracy = accuracy_score(t_label.data.cpu(),pre.data.cpu())
    precision = precision_score(t_label.data.cpu(),pre.data.cpu(), zero_division = 0)
    recall = recall_score(t_label.data.cpu(),pre.data.cpu(), zero_division = 0)
    f1 = f1_score(t_label.data.cpu(),pre.data.cpu(), zero_division = 0)
    # fpr,tpr,_ = roc_curve(t_label.data.cpu(),predict.data.cpu())
    auc = roc_auc_score(t_label.data.cpu(),predict.data.cpu())

    return precision, recall, f1, accuracy, auc 

In [42]:
def test():
    with torch.no_grad():
        model.eval()
        graphs,features,node_degs,graph_sizes = merge_Graph(getGraphList(test_graph_data))
        graphs = graphs.cuda()
        labels = torch.FloatTensor(test_label).cuda()
        node_degs = node_degs.cuda()
        features = features.cuda()
        edges = getEdgeList(test_edge_data,sortk)
        edges = edges.cuda()
            
        graphs = Variable(graphs)
        node_degs = Variable(node_degs)
        features = Variable(features)
        edges = Variable(edges)
            
        output = model(features,graphs,node_degs,graph_sizes,edges)
        l = loss(output,labels)
        precesion,recall,f1_score,acc,auc = test_evaluate(torch.sigmoid(output),labels,threshold)

    return precesion,recall,f1_score,acc,auc

def valid():
    with torch.no_grad():
        model.eval()
        graphs,features,node_degs,graph_sizes = merge_Graph(getGraphList(valid_graph_data))
        graphs = graphs.cuda()
        labels = torch.FloatTensor(valid_label).cuda()
        node_degs = node_degs.cuda()
        features = features.cuda()
        edges = getEdgeList(valid_edge_data,sortk)
        edges = edges.cuda()
                
        graphs = Variable(graphs)
        node_degs = Variable(node_degs)
        features = Variable(features)
        edges = Variable(edges)
            
        output = model(features,graphs,node_degs,graph_sizes,edges)
        l = loss(output,labels)
        precesion,recall,f1_score,acc = evaluate(torch.sigmoid(output),labels,threshold)

    return precesion,recall,f1_score, acc, l.item()

def train(epoch):
    model.train()
    start = time()
    accuracy = []
    train_precesion = []
    train_recall = []
    train_f1 = []
    j = 0
    global early_stop_flag
    for graph_files,edge_files,labels in zip([train_graph_data[i:i + batch_size] for i in range(0, len(train_graph_data), batch_size)],[train_edge_data[i:i + batch_size] for i in range(0, len(train_edge_data), batch_size)],[train_label[i:i + batch_size] for i in range(0, len(train_label), batch_size)]):
#         print('-',end = ' ')
        j += 1
#         print(files,labels)
        ss = time()
        graphs,features,node_degs,graph_sizes = merge_Graph(getGraphList(graph_files))
        graphs = graphs.cuda()
        labels = torch.FloatTensor(labels).cuda()
        node_degs = node_degs.cuda()
        features = features.cuda()
        edges = getEdgeList(edge_files,sortk)
        edges = edges.cuda()
        
        graphs = Variable(graphs)
        node_degs = Variable(node_degs)
        features = Variable(features)
        edges = Variable(edges)
        
        
        output = model(features,graphs,node_degs,graph_sizes,edges)
        l = loss(output,labels)
        optimizer.zero_grad()
        l.backward()
#         for name, parms in model.named_parameters():	
#             print('-->name:', name, '-->grad_requirs:',parms.requires_grad, \
#              ' -->grad_value:',parms.grad)    #查看梯度
        optimizer.step()
        scheduler.step()
        
        pre,rec,f1, acc = evaluate(output,labels,threshold)
        accuracy.append(acc)
#         train_precesion.append(pre)
#         train_recall.append(rec)
#         train_f1.append(f1)
        
        ee = time()
#     train_precesion = np.array(precesion).mean()
#     train_recall = np.array(recall).mean()
#     train_f1 = np.array(f1_score).mean() 
    accuracy = np.array(accuracy).mean()
    test_precesion,test_recall,test_f1,tacc, tloss = valid()
    early_stopping(tloss, model)
    if early_stopping.early_stop:
        early_stop_flag = True
    end = time()
    f = open(wf+'records.txt','a') 
    print('epoch %d, train_loss : %f  test_loss : %f trainacc: %f testacc: %f precesion: %f  recall: %f  f1_score: %f  time: %f' % (epoch+1, l.item(), tloss, accuracy, tacc, test_precesion, test_recall, test_f1, end-start),file=f)

In [43]:
# output = torch.FloatTensor([0.2,0.6,0.7,0.1,0.3])
# label = torch.FloatTensor([1,0,0,0,0])
# # a = np.array(output)
# # [1.0 if i > 0 else 0.0 for i in output] == label
# ((output>0.6).float() == label).float().sum().item()

In [44]:
# 60%  k = 67,  80% = 99
# classNum,dropout_rate,nfeat,nhid,nclass,n_layers,k,head,features_length
model = Classifier(
    classNum = 1,
    dropout_rate=0.5,
    nfeat = 100, 
    nhid = 11, 
    nclass = 1, 
    n_layers = 9, 
    k = 200,
    head = 2,
    features_length = 100
)
sortk = 200
batch_size = 32  
threshold = 0.45
if torch.cuda.is_available():
    model.cuda()
    
# loss = nn.CrossEntropyLoss()  
loss = nn.BCEWithLogitsLoss()    # This loss combines a Sigmoid layer and the BCELoss in one single class.
# optimizer = optim.SGD(model.parameters(),lr = 0.001)
optimizer = optim.Adam(model.parameters(),lr = 0.1)
scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.0004 ,cycle_momentum = False)

patience = 25 # When the loss of on validation set did not decrease in 20 consecutive training cycles, the model training was stopped to prevent model overfitting
early_stopping = EarlyStopping(patience, verbose=True)

model.parameters

<bound method Module.parameters of Classifier(
  (resGCNN): res_GCN(
    (layers): ModuleList(
      (0): GraphConvolution (100 -> 11)
      (1): GraphConvolution (11 -> 11)
      (2): GraphConvolution (11 -> 11)
      (3): GraphConvolution (11 -> 11)
      (4): GraphConvolution (11 -> 11)
      (5): GraphConvolution (11 -> 11)
      (6): GraphConvolution (11 -> 11)
      (7): GraphConvolution (11 -> 11)
      (8): GraphConvolution (11 -> 11)
      (9): GraphConvolution (11 -> 1)
    )
  )
  (edgeAttention): Edge_Attention(
    (Attention): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
      )
      (linear1): Linear(in_features=100, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=100, bias=True)
      (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm(

In [45]:
train_loss = []
test_acc = []
train_acc = []
test_loss = []
l_rs = []
Precesion = []
Recall = []
F1_score = []
AUC = []
wf = './BFS_EA_RGCN(SG)/'
early_stop_flag = False

In [46]:
def start_training():
    global train_graph_data,train_edge_data,train_label,early_stop_flag,model
    for j in range(500):
        # 每个epoch打乱训练集数据
        if early_stop_flag:
            break
        train_graph_data,train_edge_data,train_label = shuffle(train_graph_data,train_edge_data,train_label)
        train(j)
    # 获得 early stopping 时的模型参数
    model.load_state_dict(torch.load('checkpoint.pt'))

In [47]:
for i in tqdm(range(30)):
    s_time = time()
    # 打乱数据
    graph_filename,edge_filename,all_label = shuffle(graph_filename,edge_filename,all_label)

    #划分数据集
    train_graph_data = graph_filename[0:int(k*length)]
    train_label = all_label[0:int(k*length)]
    train_edge_data = edge_filename[0:int(k*length)]

    valid_graph_data = graph_filename[int(k*length):int(k1*length)]
    valid_label = all_label[int(k*length):int(k1*length)]
    valid_edge_data = edge_filename[int(k*length):int(k1*length)]

    test_graph_data = graph_filename[int(k1*length):]
    test_label = all_label[int(k1*length):]
    test_edge_data = edge_filename[int(k1*length):]
    
    model = Classifier(
    classNum = 1,
    dropout_rate=0.5,
    nfeat = 100, 
    nhid = 11, 
    nclass = 1, 
    n_layers = 9, 
    k = 200,
    head = 2,
    features_length = 100
    )
    if torch.cuda.is_available():
        model.cuda()

    # loss = nn.CrossEntropyLoss()  
    loss = nn.BCEWithLogitsLoss()    # This loss combines a Sigmoid layer and the BCELoss in one single class.
    # optimizer = optim.SGD(model.parameters(),lr = 0.001)
    optimizer = optim.Adam(model.parameters(),lr = 0.1)
    scheduler = optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.0004 ,cycle_momentum = False)
    early_stopping = EarlyStopping(patience, verbose=True) # 关于 EarlyStopping 的代码可先看博客后面的内容
    early_stop_flag = False
    start_training()
    
    pre,recall,f1,acc,auc = test()
    e_time = time()
    f = open(wf+'test_result.txt','a')
    print('exp-%d acc: %f  precesion: %f  recall: %f  f1_score: %f auc: %f\n' % (i,acc, pre, recall, f1,auc),file = f)
    print('exp-%d acc: %f  precesion: %f  recall: %f  f1_score: %f auc: %f time: %f' % (i,acc, pre, recall, f1,auc, e_time - s_time))
    test_acc.append(acc)
    Precesion.append(pre)
    Recall.append(recall)
    F1_score.append(f1)
    AUC.append(auc)
    f.close()
    
f = open(wf+'test_result.txt','a')
print('ave_acc: %f  ave_precesion: %f  ave_recall: %f  ave_f1_score: %f ave_auc: %f\n' % (np.array(test_acc).mean(), np.array(Precesion).mean(), np.array(Recall).mean(), np.array(F1_score).mean(),np.array(AUC).mean()),file=f)
print('var_acc: %f  var_precesion: %f  var_recall: %f  var_f1_score: %f var_auc: %f\n' % (np.array(test_acc).var(), np.array(Precesion).var(), np.array(Recall).var(), np.array(F1_score).var(),np.array(AUC).var()),file=f)
f.close()

  0%|          | 0/10 [00:00<?, ?it/s]

Validation loss decreased (inf --> 0.203829).  Saving model ...
Validation loss decreased (0.203829 --> 0.177505).  Saving model ...
Validation loss decreased (0.177505 --> 0.171084).  Saving model ...
Validation loss decreased (0.171084 --> 0.170744).  Saving model ...
Validation loss decreased (0.170744 --> 0.166787).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (0.166787 --> 0.157121).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (0.157121 --> 0.143855).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
EarlyStopping counter: 8 out of 25
EarlyStopping counter: 9 out of 25
EarlyStopping counter: 10 out of 25
EarlyStopping counter: 11 out of 25
EarlyStopping counter: 12 out 

 10%|█         | 1/10 [51:01<7:39:17, 3061.98s/it]

exp-0 acc: 0.945000  precesion: 0.939252  recall: 0.957143  f1_score: 0.948113 auc: 0.993108 time: 3061.978524
Validation loss decreased (inf --> 0.182422).  Saving model ...
EarlyStopping counter: 1 out of 25
Validation loss decreased (0.182422 --> 0.178823).  Saving model ...
Validation loss decreased (0.178823 --> 0.171028).  Saving model ...
Validation loss decreased (0.171028 --> 0.142370).  Saving model ...
Validation loss decreased (0.142370 --> 0.133693).  Saving model ...
Validation loss decreased (0.133693 --> 0.132945).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
Validation loss decreased (0.132945 --> 0.123256).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
Validation loss decreased (0.123256 --> 0.120036).  Saving mo

 20%|██        | 2/10 [1:25:50<6:09:20, 2770.08s/it]

...

exp-7 acc: 0.917500  precesion: 0.886878  recall: 0.960784  f1_score: 0.922353 auc: 0.979154 time: 1399.287441
Validation loss decreased (inf --> 0.159219).  Saving model ...
Validation loss decreased (0.159219 --> 0.149218).  Saving model ...
Validation loss decreased (0.149218 --> 0.138789).  Saving model ...
Validation loss decreased (0.138789 --> 0.124273).  Saving model ...
Validation loss decreased (0.124273 --> 0.108924).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
Validation loss decreased (0.108924 --> 0.107031).  Saving model ...
Validation loss decreased (0.107031 --> 0.099982).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 o

 90%|█████████ | 9/10 [5:22:02<32:06, 1926.27s/it]  

exp-8 acc: 0.932500  precesion: 0.949749  recall: 0.917476  f1_score: 0.933333 auc: 0.985312 time: 1672.402855
Validation loss decreased (inf --> 0.172137).  Saving model ...
Validation loss decreased (0.172137 --> 0.160355).  Saving model ...
Validation loss decreased (0.160355 --> 0.147182).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased (0.147182 --> 0.146316).  Saving model ...
Validation loss decreased (0.146316 --> 0.136702).  Saving model ...
Validation loss decreased (0.136702 --> 0.133493).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
EarlyStopping counter: 3 out of 25
EarlyStopping counter: 4 out of 25
EarlyStopping counter: 5 out of 25
EarlyStopping counter: 6 out of 25
EarlyStopping counter: 7 out of 25
Validation loss decreased (0.133493 --> 0.122487).  Saving model ...
EarlyStopping counter: 1 out of 25
EarlyStopping counter: 2 out of 25
Validation loss decreased 

100%|██████████| 10/10 [5:51:36<00:00, 2109.62s/it]

exp-9 acc: 0.917500  precesion: 0.898990  recall: 0.931937  f1_score: 0.915167 auc: 0.986347 time: 1773.305332


In [48]:
# for name, parms in model.named_parameters():	
#     print('-->name:', name, '-->grad_requirs:',parms.requires_grad, \
#              ' -->grad_value:',parms.grad)    #查看梯度

In [49]:
torch.save(model, wf+'BFS_EA_RGCN.pkl')

In [50]:
# a = np.loadtxt('./BFS_EA_RGCN/500_bat5_roc_curve.txt')
# b = np.loadtxt('./DFS_EA_RGCN/500_roc_curve.txt')
# c = np.loadtxt('./GCN/500_roc_curve.txt')
# d = np.loadtxt('./BFS_EA_GCN/500_roc_curve.txt')

# plt.title('Roc Curve')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.xlim([-0.05,1])
# plt.ylim([-0.05,1.05])
# plt.plot(a[0],a[1],'b-')
# plt.plot(b[0],b[1],'g-')
# plt.plot(c[0],c[1],'r-')
# plt.plot(d[0],d[1],'c-')

# plt.legend(['BFS_EA_RGCN | auc=0.9344','DFS_EA_RGCN | auc=0.9188','GCN | auc=0.8886','BFS_EA_GCN | auc = 0.9296'])

# plt.savefig(wf+'cmp_roc.png')